# **Introducción al análisis de datos en Python** 
#### Profesora: Catalina Bernal

### Unir bases de datos (`merge`)

Ya aprendimos cómo concatenar filas y columnas de diferentes bases de datos. Para hacer esto es necesario que la cantidad de columnas y filas, respectivamente, de los dataframes a juntar sean los mismos y que sus índices o llaves también lo sean.

No obstante, muchas veces cuando trate de juntar bases de datos, notará que no necesariamente todas las llaves están presentes en ambas bases de datos, o que incluso, a cada fila de la base izquierda, querrá pegarle más de una fila de la base derecha, o viceversa.

A la hora de hacer pegues más complejos, hablamos de que vamos a utilizar un `merge`. 

Comencemos con la sintaxis del `merge`. Para pegar dos bases de datos, usted usará un comando similar al siguiente:

```python
pd.merge(left = left_dataframe, right = right_dataframe, on = "alguna(s)_columa(s)", how = "left|right|inner|outer")
```

Los argumentos que toma la función son:
- `left`: dataframe que va de primero.
- `right`: dataframe que va de segundo.
- `on`: es la columna o la lista de columnas que determinan qué filas de una tabla coinciden con qué filas de la segunda tabla. Comúnmente a estas variables se les llaman las llaves del pegue y debe identificar a cada observación de forma única. A veces, las columnas que desea fusionar tienen nombres diferentes en los datos. Por ejemplo, suponga que tiene dos bases de datos, una que registra el dinero mensual gastado por persona en almacenes Éxito y otra que tiene características personales de las personas. Usted podría tratar de juntar ambas bases con el identificador de fila o persona de cada base que en este caso podría ser la cédula, sin embargo, en un dataframe tal vez la variable se llame "cc" mientras que en el otro puede que se llame "cédula". En esos casos, puede especificar los nombres de columna por separado para cada marco de datos utilizando los argumentos "left_on" y "right_on".
- `how`: es el método a usar, por defecto Pandas usa el método "inner". Más adelante exploraremos más al respecto.

<center>
<div>
<img src="./img/merges.png" width="400"/>
</div>
</center>

Tenemos cuatro grandes métodos para relacionar las bases porque no siempre tenemos una coincidencia uno a uno (one to one) entre las filas. Estos cuatro métodos afectan la forma en que Pandas trata los datos no coincidentes y eso es lo que veremos más adelante.

<center>
<div>
<img src="./img/one-many.png" width="400"/>
</div>
</center>



In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# ejemplos de pegues
left_dataframe = pd.DataFrame({"ID": [1,2,3,4], "left_side": "Izquierda"})
right_dataframe = pd.DataFrame({"ID": [3,4,5,6], "right_side": "Derecha"})

In [ ]:
left_dataframe

In [ ]:
right_dataframe

#### Left merge
En un Left merge lo que más nos interesa son los datos del lado IZQUIERDO a los cuales queremos pegarles columnas de una base de datos en el lado DERECHO.

Para hacer eso, cortamos las filas en el marco de datos DERECHO y pegamos partes en el marco de datos IZQUIERDO. Recuerde, nos preocupamos principalmente por el lado IZQUIERDO y solo queremos datos del lado DERECHO si tiene alguna de las mismas ID. Entonces, si algo en el marco de datos DERECHO no coincide o no existe, entonces tenemos que hacer cosas para mantener las columnas de la misma longitud. Lo hacemos agregando NaN para llenar el vacío o descartando algunas filas por completo.

En este ejemplo, el lado IZQUIERDO tiene los ID 1, 2, 3 y 4:
- El lado DERECHO no tiene ID 1 o 2, por lo que agregamos NaN porque necesitamos que las columnas tengan la misma longitud.
- El lado DERECHO tiene datos para los ID 3 y 4, así que lo agregamos como una nueva columna.
- El lado IZQUIERDO no tiene ID 5 o 6, por lo que no necesitamos esa información del DERECHO y se descarta.

<center>
<div>
<img src="./img/left_merge.png" width="400"/>
</div>
</center>

In [ ]:
# Left merge con "ID" como llave
pd.merge(left = left_dataframe, right = right_dataframe, on = "ID", how = "left")

#### Right merge
Los Right merges funcionan igual que los Left merges, la diferencia es que nos preocupamos principalmente por el lado DERECHO y nos gustaría agregar datos desde el IZQUIERDO si tienen ID coincidentes.

<center>
<div>
<img src="./img/right_merge.png" width="400"/>
</div>
</center>

In [ ]:
# Right merge con "ID" como llave
pd.merge(left = left_dataframe, right = right_dataframe, on = "ID", how = "right")

#### Inner merge
Con un Inner merge, cortamos ambos marcos de datos y solo pegamos las cosas que coinciden. Si una ID no está en ambos marcos de datos, no la mantenemos y no agregamos NaN.

<center>
<img src="./img/inner_merge.png" width="400"/>
</center>

In [ ]:
# Inner merge con "ID" como llave
pd.merge(left = left_dataframe, right = right_dataframe, on = "ID", how = "inner")

#### Outer merge
Con un Outer merge, cortamos ambos marcos de datos y mantenemos todo de ambos lados. Luego agregamos NaN para llenar los espacios en blanco.

<center>
<img src="./img/outer_merge.png" width="400"/>
</center>

In [ ]:
# Outer merge con "ID" como llave
pd.merge(left = left_dataframe, right = right_dataframe, on = "ID", how = "outer")

In [ ]:
# Creamos dos DataFrames: empleados y departamentos
empleados = pd.DataFrame({
    'id': [1, 2, 3, 4],
    'nombre': ['Ana', 'Luis', 'Carlos', 'Sofía'],
    'departamento_id': [10, 20, 10, 30]
})

departamentos = pd.DataFrame({
    'departamento_id': [10, 20, 30],
    'departamento': ['Ventas', 'Marketing', 'TI']
})

# 🔗 Unimos usando la columna en común
df_unido = empleados.merge(departamentos, on='departamento_id')
print(df_unido)

In [ ]:
# Queremos saber si el nombre de la persona empieza con vocal
def empieza_con_vocal(nombre):
    return nombre[0].lower() in 'aeiou'

empleados['empieza_vocal'] = empleados['nombre'].apply(empieza_con_vocal)
print(empleados)

### Groupby


Uno de los métodos más útiles para los analistas de datos es `.groupby()`. Este método permite dividir los datos en grupos y a cada uno de estos aplicarles una función de agregación.

Veamos el siguiente ejemplo para entender este concepto mejor:

In [ ]:
df = pd.read_excel(f"data/ejemplo_groupby.xlsx")
df

In [ ]:
df.animal.unique()

Note que tenemos un `dataframe` con cuatro tipos de animales: 
- alligators (cocodrilos 🐊)
- cats (gatos 🐱)
- snakes (serpientes 🐍)
- hamsters (hamsters 🐹)

Cada una de las filas indican un chequeo en el veterinario donde se registra edad, peso y largo del animal. Por ende, usted como investigador quiere estudiar algunas estadísticas descriptivas por especie. Por ejemplo ¿Cuál es el peso promedio de cada especie?

In [ ]:
# El primer paso es agrupar por animal
animal_groups = df.groupby("animal")

In [ ]:
animal_groups

In [ ]:
# Veamos la conformación de cada uno de los grupos. ¿En qué filas aparece cada animal?
animal_groups.groups

Visualmente, lo que sucedió fue lo siguiente:

1. Se agrupa los valores únicos de la columna animal.
<center>
<img src = "./img/groupby1.jpg" width = "400">
</center>

2. La segmentación de cada grupo se vería de la siguiente manera
<center>
<img src = "./img/groupby2.jpg" width = "400">
</center>

3. Se le asignan las otras variables/columnas a cada grupo
<center>
<img src = "./img/groupby3.jpg" width = "400">
</center>

4. Se aplica la función agregadora `.mean()` sobre la columna `weight` de cada grupo.
<center>
<img src = "./img/groupby4.jpg" width = "400">
</center>


In [ ]:
df

#### Método .agg()
El método .agg() se puede utilizar después de aplicar un método .groupby() en pandas para realizar operaciones de agregación en los datos de cada grupo.

La sintaxis general de la función .groupby() es la siguiente:
```python
dataframe.groupby(columnas).agg(funciones)
```
Donde:
- dataframe: el DataFrame al que se aplicará la función `groupby()`.
- columnas: la(s) columna(s) que se utilizarán para agrupar los datos.
- funciones: la(s) operación(es) de agregación que se aplicarán a los datos agrupados.

Por ejemplo, para calcular la media, el máximo y el mínimo de las columnas de peso y longitud del DataFrame agrupado por la columna 'animal', se puede utilizar la siguiente sintaxis:

In [ ]:
df.groupby("animal")[['weight','length']].agg(["min", "mean", "max"])

In [ ]:
df.groupby("animal").agg({'weight': ['mean', 'max'], 'length': 'std', 
                                     "age": lambda x: np.percentile(x, 50)})

In [ ]:
# Otra sintaxis, en vez de un diccionario, usar tuplas
# (nombre_columna,funcion)
df.groupby("animal").agg(peso_promedio = ("weight", 'mean'), 
                                   peso_maximo = ("weight", 'max'),
                                   edad_mediana = ("age", lambda x: np.percentile(x, 50)))

## Ejercicio

### Información de accidentes de tránsito en Bogotá en el 2016
A partir de la base de datos `info_accidentes.csv`, responda las siguientes preguntas:

1. ¿Cuántos muertos hay registrados en la base de datos?
2. ¿Cuántos heridos hay registrados en la base de datos?
3. ¿Cuantos accidentes hubo en la localidad de Santa Fe? ¿Cuántos muertos y cuántos heridos dejaron estos accidentes en total?
4. Explore la columna TipoTiempo, ¿cuál es la categoría en la que más ocurren accidentes?